## --- Day 9: Smoke Basin ---

These caves seem to be lava tubes. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:
```
2199943210
3987894921
9856789892
8767896789
9899965678
```
Each number corresponds to the height of a particular location, where 9 is the highest and 0 is the lowest a location can be.

Your first goal is to find the low points - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are four low points, all highlighted: two are in the first row (a 1 and a 0), one is in the third row (a 5), and one is in the bottom row (also a 5). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?


In [1]:
def find_low_points(heightmap):
    def _min_adjacent(r, c):
        up, left, right, down = 10, 10, 10, 10
        if r > 0:
            up = heightmap[r-1][c]
        if c > 0:
            left = heightmap[r][c-1]
        if c < len(heightmap[r])-1:
            right = heightmap[r][c+1]
        if r < len(heightmap)-1:
            down = heightmap[r+1][c]
        # print(f"adjacent to {heightmap[r][c]} {r},{c}:", up, left, down, right, "min", min(up, left, right, down))

        return min(up, left, right, down)

    low_points = []

    for r in range(len(heightmap)):
        for c in range(len(heightmap[r])):
            if heightmap[r][c] < _min_adjacent(r, c):
                low_points.append((r, c))
    
    return low_points

def sum_risk_levels(heightmap, low_points):
    result = 0

    for r, c in low_points:
        result += heightmap[r][c] + 1
    
    return result

def map_risk_levels(heightmap):
    low_points = find_low_points(heightmap)

    return sum_risk_levels(heightmap, low_points)

In [2]:
example1_input = [
    [2, 1, 9, 9, 9, 4, 3, 2, 1, 0],
    [3, 9, 8, 7, 8, 9, 4, 9, 2, 1],
    [9, 8, 5, 6, 7, 8, 9, 8, 9, 2],
    [8, 7, 6, 7, 8, 9, 6, 7, 8, 9],
    [9, 8, 9, 9, 9, 6, 5, 6, 7, 8]
]
example1_solution = 15

assert example1_solution == map_risk_levels(example1_input)

In [3]:
def read_file(filename):
    rows = []

    with open(filename) as f:
        for line in f.readlines():
            str_row = list(line.strip())
            rows.append([int(s) for s in str_row])

    return rows

problem1_input = read_file("./inputs/Day9.txt")

assert len(problem1_input) == 100


In [4]:
# Problem 1 solution
map_risk_levels(problem1_input)

575

## --- Part Two ---

Next, you need to find the largest basins so you know what areas are most important to avoid.

A basin is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height 9 do not count as being in any basin, and all other locations will always be part of exactly one basin.

The size of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size 3:
```
2199943210
3987894921
9856789892
8767896789
9899965678
```
The top-right basin, size 9:
```
2199943210
3987894921
9856789892
8767896789
9899965678
```
The middle basin, size 14:
```
2199943210
3987894921
9856789892
8767896789
9899965678
```
The bottom-right basin, size 9:
```
2199943210
3987894921
9856789892
8767896789
9899965678
```
Find the three largest basins and multiply their sizes together. In the above example, this is 9 * 14 * 9 = 1134.

What do you get if you multiply together the sizes of the three largest basins?


In [5]:
def map_basins(heightmap, low_points):
    # Map of each low point to a set of all points in the low point's basin
    basins = {}

    def _add_point_to_basin(point, low_point):
        # If point (r, c) is a 9 or out of bounds, can't add any more
        r, c = point
        if r < 0 or r >= len(heightmap) or c < 0 or c >= len(heightmap[r]) or heightmap[r][c] == 9:
            return
        
        # Add the point to low_point's basin if it has not already been added
        if low_point not in basins:
            # Create the set if it's not there
            basins[low_point] = set()
        elif point in basins[low_point]:
            # This point was already added so we're done
            return

        # Add this point to the set
        # print(f"add lp {low_point} update ", r, c)
        basins[low_point].update([(r, c)])

        # Then try to add adjacent points to the left/right/up/down recursively
        _add_point_to_basin((r, c-1), low_point)
        _add_point_to_basin((r, c+1), low_point)
        _add_point_to_basin((r-1, c), low_point)
        _add_point_to_basin((r+1, c), low_point)

        return

    for low_point in low_points:
        _add_point_to_basin(low_point, low_point)

    return basins

def find_basin_sizes(heightmap):
    low_points = find_low_points(heightmap)
    basins = map_basins(heightmap, low_points)

    return sorted([len(s) for s in basins.values()])

In [6]:
example2_solution = 1134
ex2_basins = find_basin_sizes(example1_input)

assert example2_solution == ex2_basins[-1] * ex2_basins[-2] * ex2_basins[-3]

In [7]:
# Part 2 - same input
basins = find_basin_sizes(problem1_input)
basins[-1] * basins[-2] * basins[-3] 

1019700